In [ ]:
#!pip install GitPython
#!pip install sagemaker
#!pip install s3fs
#!pip install aws-mlops
import os
import sys
import pandas as pd
import config as config
sys.path.insert(1, os.getcwd() + '/..')
from aws_mlops.data_storage import DataStorage
from sklearn.model_selection import train_test_split

In [ ]:
# object for data storage management
studio = DataStorage(config.source_bucket, config.test_data_key)
config_name = 'auto_input'

In [ ]:
# load dataframe the first time
# docs: https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality.names
#df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', sep=';')
#studio.checkpoint(df, config.raw_data_filename, f's3://{config.source_bucket}/{config.raw_data_key}')
#df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv', sep=';')
#studio.checkpoint(df, config.new_data_filename, f's3://{config.source_bucket}/{config.new_data_key}')

# load dataframe after loading on s3
df = studio.restore(config.raw_data_filename, f's3://{config.source_bucket}/{config.raw_data_key}')

In [ ]:
# prep data: it is indifferent the target column position
df.insert(0, 'id', range(883, 883 + len(df)))
train, test, _, _ = train_test_split(df, df[config.target], test_size=config.test_size, random_state=0)
train.drop('id', axis = 1, inplace = True)

In [ ]:
# output - columns names, target and identifier columns
test = studio.save_test(test, [config.target, config.identifier])

In [ ]:
# saving dataframes
output = getattr(config, config_name)['outputs'][0]
# train with header
studio.save_on_s3(train, output['OutputName'], output['S3Output']['S3Uri'], header = True, index = False)
#studio.local_save(train, output['S3Output']['LocalPath'], output['OutputName'], header = True, index = False)
output = getattr(config, config_name)['outputs'][1]
# test without header
studio.save_on_s3(test, output['OutputName'], output['S3Output']['S3Uri'], header = False, index = False)
#studio.local_save(test, output['S3Output']['LocalPath'], output['OutputName'], header = False, index = False)